In [1]:
!pip install faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 17.7 MB/s eta 0:00:00


In [2]:
# import pandas as pd
# from sentence_transformers import SentenceTransformer, util
# import faiss
# import numpy as np
# from transformers import pipeline
# from sklearn.metrics import precision_score, recall_score

# df = pd.read_json("hf://datasets/MongoDB/airbnb_embeddings/airbnb_embeddings.json", lines=True)
# print("Columns in the dataset:", df.columns)

# descriptions = df['description'].tolist()

# embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
# embeddings = embedding_model.encode(descriptions, convert_to_tensor=True).cpu().detach().numpy()
# labels = df['accessibility_label'].tolist() if 'accessibility_label' in df.columns else ["default_label"] * len(df)


# d = embeddings.shape[1]  # ממד של ה-embeddings
# index = faiss.IndexFlatL2(d)
# index.add(embeddings.astype('float32'))  # ודא שה-embeddings הם מסוג float32

# def retrieve_properties(query, k=10, filter_labels=None):  # שינוי k ל-10
#     query_embedding = embedding_model.encode(query, convert_to_tensor=True).cpu().detach().numpy()
#     query_embedding = np.expand_dims(query_embedding, axis=0).astype('float32')  # הוספת ממד נוסף כדי להתאים ל-FAISS
#     distances, indices = index.search(query_embedding, k)
#     results = []

#     for idx in indices[0]:
#         results.append(descriptions[idx])

#     print("Retrieved descriptions:", results)
#     return results

# generator = pipeline("text-generation", model="distilgpt2")

# def generate_response(query, filter_labels=None):
#     relevant_descriptions = retrieve_properties(query, filter_labels=filter_labels)
#     if not relevant_descriptions:
#         return "No relevant descriptions found."

#     prompt = "Based on these property descriptions, describe a suitable property for a user with special needs:\n"
#     prompt += "\n".join(relevant_descriptions)
#     response = generator(prompt, max_new_tokens=50, num_return_sequences=1, truncation=True)
#     return response[0]['generated_text']


# def rerank_results(results, preferences):
#     if not results:
#         return results

#     ranked_results = []
#     for result in results:
#         score = sum(1 for word in preferences if word in result.lower())
#         ranked_results.append((result, score))
#     ranked_results.sort(key=lambda x: x[1], reverse=True)
#     return [res[0] for res in ranked_results]

# def evaluate_rag_system(true_answers, generated_answers):
#     precision = precision_score(true_answers, generated_answers, average='binary')
#     recall = recall_score(true_answers, generated_answers, average='binary')
#     print(f"Precision: {precision:.2f}")
#     print(f"Recall: {recall:.2f}")

# # --- דוגמה לשימוש בפונקציות ---

# query = "Looking for a wheelchair accessible property with an elevator."

# retrieved_properties = retrieve_properties(query, k=10, filter_labels=None)  # שינוי ל-k=10 וללא סינון

# response = generate_response(query, filter_labels=None)
# print("Generated Response:", response)

# preferences = ["wheelchair", "elevator", "accessible"]
# ranked_results = rerank_results(retrieved_properties, preferences)
# print("Ranked Results:", ranked_results)

# true_answers = [1, 0, 1, 1, 0]  # תשובות אמיתיות
# generated_answers = [1, 0, 1, 0, 0]  # תשובות שנוצרו
# evaluate_rag_system(true_answers, generated_answers)


In [3]:
# import pandas as pd
# from sentence_transformers import SentenceTransformer
# import faiss
# import numpy as np
# from transformers import pipeline
# from sklearn.metrics import precision_score, recall_score

# df = pd.read_json("hf://datasets/MongoDB/airbnb_embeddings/airbnb_embeddings.json", lines=True)
# print("Columns in the dataset:", df.columns)

# descriptions = df['description'].tolist()
# embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
# embeddings = embedding_model.encode(descriptions, convert_to_tensor=True).cpu().detach().numpy()

# d = embeddings.shape[1]
# index = faiss.IndexFlatL2(d)
# index.add(embeddings.astype('float32'))

# #  שליפה מבוססת שאילתות
# def retrieve_properties(query, k=10):
#     query_embedding = embedding_model.encode(query, convert_to_tensor=True).cpu().detach().numpy()
#     query_embedding = np.expand_dims(query_embedding, axis=0).astype('float32')
#     distances, indices = index.search(query_embedding, k)
#     return [descriptions[idx] for idx in indices[0]]


# generator = pipeline("text-generation", model="distilgpt2")

# def generate_response(query):
#     relevant_descriptions = retrieve_properties(query, k=5)
#     if not relevant_descriptions:
#         return "No relevant descriptions found."

#     prompt = "Based on these property descriptions, describe a suitable property:\n"
#     prompt += "\n".join(relevant_descriptions[:3])  # הגבלת מספר התיאורים ל-3

#     response = generator(prompt, max_new_tokens=50, num_return_sequences=1, truncation=True)
#     return response[0]['generated_text']


# def rerank_results(results, preferences):
#     ranked_results = sorted(results, key=lambda x: sum(1 for word in preferences if word in x.lower()), reverse=True)
#     return ranked_results


# def evaluate_rag_system(true_answers, generated_answers):
#     precision = precision_score(true_answers, generated_answers, average='binary')
#     recall = recall_score(true_answers, generated_answers, average='binary')
#     print(f"Precision: {precision:.2f}")
#     print(f"Recall: {recall:.2f}")

# #  דוגמה לשימוש
# query = "Looking for a wheelchair accessible property with an elevator."
# retrieved_properties = retrieve_properties(query, k=10)
# response = generate_response(query)
# print("Generated Response:", response)

# preferences = ["wheelchair", "elevator", "accessible"]
# ranked_results = rerank_results(retrieved_properties, preferences)
# print("Ranked Results:", ranked_results[:5])

# true_answers = [1, 0, 1, 1, 0]
# generated_answers = [1, 0, 1, 0, 0]
# evaluate_rag_system(true_answers, generated_answers)


# שילוב שלב שלישי ורביעי

In [4]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from transformers import pipeline
from sklearn.metrics import precision_score, recall_score

# Load the Airbnb dataset
df = pd.read_json("hf://datasets/MongoDB/airbnb_embeddings/airbnb_embeddings.json", lines=True)
print("Columns in the dataset:", df.columns)

# Load Itay's synthetic QA pairs from the CSV file
qa_df = pd.read_csv("/content/synthetic_qa_pairs_cohere_sample.csv")
print("Columns in synthetic QA file:", qa_df.columns)  # Confirm the columns

# Prepare descriptions for embedding
descriptions = df['description'].tolist()
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = embedding_model.encode(descriptions, convert_to_tensor=True).cpu().detach().numpy()

# Create FAISS index
d = embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(embeddings.astype('float32'))

# Query-based retrieval function
def retrieve_properties(query, k=10):
    query_embedding = embedding_model.encode(query, convert_to_tensor=True).cpu().detach().numpy()
    query_embedding = np.expand_dims(query_embedding, axis=0).astype('float32')
    distances, indices = index.search(query_embedding, k)
    return [descriptions[idx] for idx in indices[0]]

# Load text generation pipeline
generator = pipeline("text-generation", model="distilgpt2")

# Function to generate response using the retrieved descriptions and synthetic QA pairs
def generate_response_with_qa(query):
    # Retrieve relevant descriptions
    relevant_descriptions = retrieve_properties(query, k=5)
    if not relevant_descriptions:
        return "No relevant descriptions found."

    # Find relevant QA pairs from Itay's dataset
    relevant_qa_pairs = qa_df[qa_df['synthetic_question'].str.contains(query, case=False, na=False)]

    # Construct the prompt with descriptions and QA pairs
    prompt = "Based on these property descriptions and example Q&A, describe a suitable property:\n"
    prompt += "\n".join(relevant_descriptions[:3])  # Limit to top 3 descriptions
    prompt += "\nExample Q&A:\n"
    prompt += "\n".join([f"Q: {row['synthetic_question']}\nA: {row['synthetic_answer']}" for _, row in relevant_qa_pairs.iterrows()][:3])  # Limit to top 3 QA pairs

    # Generate the response
    response = generator(prompt, max_new_tokens=50, num_return_sequences=1, truncation=True)
    return response[0]['generated_text']

# Function to rerank results based on user preferences
def rerank_results(results, preferences):
    ranked_results = sorted(results, key=lambda x: sum(1 for word in preferences if word in x.lower()), reverse=True)
    return ranked_results

# Evaluation function for the RAG system
def evaluate_rag_system(true_answers, generated_answers):
    precision = precision_score(true_answers, generated_answers, average='binary')
    recall = recall_score(true_answers, generated_answers, average='binary')
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")

# Example usage
query = "Looking for a wheelchair accessible property with an elevator."
retrieved_properties = retrieve_properties(query, k=10)
response = generate_response_with_qa(query)
print("Generated Response with QA:", response)

preferences = ["wheelchair", "elevator", "accessible"]
ranked_results = rerank_results(retrieved_properties, preferences)
print("Ranked Results:", ranked_results[:5])

true_answers = [1, 0, 1, 1, 0]
generated_answers = [1, 0, 1, 0, 0]
evaluate_rag_system(true_answers, generated_answers)


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Columns in the dataset: Index(['_id', 'listing_url', 'name', 'summary', 'space', 'description',
       'neighborhood_overview', 'notes', 'transit', 'access', 'interaction',
       'house_rules', 'property_type', 'room_type', 'bed_type',
       'minimum_nights', 'maximum_nights', 'cancellation_policy',
       'last_scraped', 'calendar_last_scraped', 'first_review', 'last_review',
       'accommodates', 'bedrooms', 'beds', 'number_of_reviews', 'bathrooms',
       'amenities', 'price', 'security_deposit', 'cleaning_fee',
       'extra_people', 'guests_included', 'images', 'host', 'address',
       'availability', 'review_scores', 'reviews', 'weekly_price',
       'monthly_price', 'text_embeddings', 'image_embeddings'],
      dtype='object')
Columns in synthetic QA file: Index(['_id', 'synthetic_question', 'synthetic_answer'], dtype='object')


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Response with QA: Based on these property descriptions and example Q&A, describe a suitable property:
Fully Accessible private room (for Physically Disabled) in Western Sydney with the following facilities: 1. Hoist for Bed / Wheelchair transfers 2. Hospital grade height adjustable bed with optional pressure relief mattress 3. Reclinable Shower Chair and Fully accessible bathroom 4. Fully accessible access ramp 5. Access to all kind of carers on short notice 6. Wheelchair accessible Van with driver (extra charges) 7. Extra single bed so the room sleeps 2 The private room is in a fully accessible house (the pool is not wheelchair accessible). The room is set up to take care of physically disabled patrons even with high-level spinal injuries. I am always there and love socialising. However, we are happy to give privacy if the interaction is not sought. The house is in a quiet and very accessible suburb in Western Sydney. The house is close to M4 & M7. Westfield shopping plaza a

In [7]:
from sklearn.metrics import precision_score, recall_score, f1_score

def evaluate_rag_system(true_answers, generated_answers):
    """
    Evaluate the RAG system's performance using precision, recall, and F1-score metrics.

    Args:
        true_answers (list): A list of ground truth binary labels (1 for relevant, 0 for non-relevant).
        generated_answers (list): A list of predicted binary labels by the RAG system.

    Prints:
        Precision, Recall, and F1-Score.
    """
    precision = precision_score(true_answers, generated_answers, average='binary')
    recall = recall_score(true_answers, generated_answers, average='binary')
    f1 = f1_score(true_answers, generated_answers, average='binary')

    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1-Score: {f1:.2f}")


# Example true answers and generated answers
true_answers = [1, 0, 1, 1, 0]
generated_answers = [1, 0, 1, 0, 0]

# Evaluate the RAG system
evaluate_rag_system(true_answers, generated_answers)


Precision: 1.00
Recall: 0.67
F1-Score: 0.80
